In [ ]:
# Oriol
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/Colab Notebooks/NLP/data/spotify_songs.csv.zip" .

In [ ]:
# Import Transformers
!pip install --upgrade transformers
!pip install simpletransformers

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import zipfile
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel

In [ ]:

# Loading data to a pandas dataframe
#with zipfile.ZipFile('data/spotify_songs.csv.zip') as zip:
with zipfile.ZipFile('spotify_songs.csv.zip') as zip:
    with zip.open('spotify_songs.csv') as myZip:
        df = pd.read_csv(myZip) 

In [ ]:
# Data preprocessing

# Filter English songs
df = df.loc[df['language'] == 'en']

# Filter relevant columns
#df = df[['track_id', 'track_name', 'track_artist', 'lyrics', 'track_album_name', 'playlist_genre']]
df = df[['lyrics', 'playlist_genre']]

# Change column names for model
df = df.rename(columns={'lyrics': 'text', 'playlist_genre': 'label'})

df.head()

In [ ]:
df['label'] = pd.factorize(df['label'])[0]
df.head(n = 20)

In [ ]:
df.head()

In [ ]:
# Split train and test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'])

# Split train and validation
train_df, validation_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'])

print(train_df.shape)
print(validation_df.shape)
print(test_df.shape)

In [ ]:
df['label'].unique()
train_df['label'].unique()

In [ ]:
# Model Arguments
model_args = ClassificationArgs(num_train_epochs=1, overwrite_output_dir=True, manual_seed=42, reprocess_input_data = True)

# Model 
model = ClassificationModel(model_type='roberta', model_name='roberta-base', use_cuda=True, num_labels=6, args=model_args)
#model = MultiLabelClassificationModel('roberta', 'roberta-base', args={'reprocess_input_data': True})

In [ ]:
# Train the model
model.train_model(train_df)

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(validation_df)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(validation_df, f1=f1_multiclass, acc=accuracy_score)

In [ ]:
print(result)

In [ ]:
predictions, raw_outputs = model.predict(['I love rock n roll'])

In [ ]:
predictions